In [1]:
import os
import pandas as pd
from fastbook import *
from fastai.tabular.all import *
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from dtreeviz.trees import *
import sklearn

def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)
def m_rmse(m, xs, y): return r_mse(m.predict(xs), y)

traindf = pd.read_csv('/mnt/c/Users/jsult/Desktop/spacetitanic3/train.csv',low_memory=False)
testdf = pd.read_csv('/mnt/c/Users/jsult/Desktop/spacetitanic3/test.csv',low_memory= False)
traindf['Set'] = 'Train'
testdf['Set'] = 'Test'
togeth = pd.concat([testdf,traindf])

ModuleNotFoundError: No module named 'pandas'

In [42]:
togeth[togeth.Cabin == 'G/2/S']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Set,Transported


In [43]:




## Need to set to categories after !!


def split_group_number(df):
    df['Group'] = df['PassengerId'].apply(lambda x: x.split("_")[0] if pd.notna(x) else pd.NA)
    df.Group = df.Group.astype('float')
    df['GroupNumber'] = df['PassengerId'].apply(lambda x: x.split("_")[1] if pd.notna(x) else pd.NA)
    df.GroupNumber = df.GroupNumber.astype('float')
    return df
togeth = split_group_number(togeth)


def split_cabin(df):
    df['CabinDeck'] = df['Cabin'].apply(lambda x: x.split("/" )[0] if pd.notna(x) else pd.NA)
    custom_order = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T']
    df['CabinDeck'] = pd.Categorical(df['CabinDeck'], categories=custom_order, ordered=True)
    df['CabinSide'] = df['Cabin'].apply(lambda x: x.split("/")[2] if pd.notna(x) else pd.NA)
    df['CabinNum'] = df['Cabin'].apply(lambda x: int(x.split("/")[1]) if pd.notna(x) else pd.NA)
    return df
togeth = split_cabin(togeth)




def split_names(df):
    # Create new columns for first name and last name
    df['FirstName'] = df['Name'].apply(lambda x: x.split()[0] if pd.notna(x) else pd.NA)
    df['LastName'] = df['Name'].apply(lambda x: x.split()[1] if pd.notna(x) else pd.NA)
    return df
togeth = split_names(togeth)

def group_size(df):
    def group_size_apply(row):
        group = df[df.Group == row.Group]
        return len(group)
    df['GroupSize'] = df.apply(group_size_apply, axis = 1)
    return df
togeth = group_size(togeth)


togeth = togeth.sort_values(by = ['Group','GroupNumber'])
togeth.reset_index(inplace = True)
togeth.drop(columns = 'index',inplace = True)


def fill_home_planet_by_last_name(df):
    for index, row in df.iterrows():
        if pd.isna(row['HomePlanet']):
            matching_rows = df[df['LastName'] == row['LastName']]
            non_nan_planet = matching_rows.dropna(subset=['HomePlanet'])
            if not non_nan_planet.empty:
                planet_to_fill = non_nan_planet['HomePlanet'].values[0]
                df.at[index, 'HomePlanet'] = planet_to_fill
    return df
def fill_home_planet_by_group(df):
    for index, row in df.iterrows():
        if pd.isna(row['HomePlanet']):
            matching_rows = df[df['Group'] == row['Group']]
            non_nan_planet = matching_rows.dropna(subset=['HomePlanet'])
            if not non_nan_planet.empty:
                planet_to_fill = non_nan_planet['HomePlanet'].values[0]
                df.at[index, 'HomePlanet'] = planet_to_fill
    return df
def home_planet_from_cabindeck_abc(df):
    df.loc[df['CabinDeck'].isin(['A', 'B', 'C']), 'HomePlanet'] = df.loc[df['CabinDeck'].isin(['A', 'B', 'C']), 'HomePlanet'].fillna('Europa')
    return df
def home_planet_from_cabindeck_g(df):
    df.loc[df['CabinDeck']== 'G', 'HomePlanet'] = df.loc[df['CabinDeck'] == 'G', 'HomePlanet'].fillna('Earth')
    return df
def homeplanet_impute(df):
    df = fill_home_planet_by_group(df)
    df = fill_home_planet_by_last_name(df)
    df = home_planet_from_cabindeck_abc(df)
    df = home_planet_from_cabindeck_g(df)
    return df
togeth = homeplanet_impute(togeth)

In [44]:
togeth[togeth.Cabin == 'G/2/S']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Set,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName,GroupSize


In [45]:
def fill_home_planet_by_last_name(df):
    for index, row in df.iterrows():
        if pd.isna(row['HomePlanet']):
            matching_rows = df[df['LastName'] == row['LastName']]
            non_nan_planet = matching_rows.dropna(subset=['HomePlanet'])
            if not non_nan_planet.empty:
                planet_to_fill = non_nan_planet['HomePlanet'].values[0]
                df.at[index, 'HomePlanet'] = planet_to_fill
    return df
def fill_home_planet_by_group(df):
    for index, row in df.iterrows():
        if pd.isna(row['HomePlanet']):
            matching_rows = df[df['Group'] == row['Group']]
            non_nan_planet = matching_rows.dropna(subset=['HomePlanet'])
            if not non_nan_planet.empty:
                planet_to_fill = non_nan_planet['HomePlanet'].values[0]
                df.at[index, 'HomePlanet'] = planet_to_fill
    return df
def home_planet_from_cabindeck_abc(df):
    df.loc[df['CabinDeck'].isin(['A', 'B', 'C']), 'HomePlanet'] = df.loc[df['CabinDeck'].isin(['A', 'B', 'C']), 'HomePlanet'].fillna('Europa')
    return df
def home_planet_from_cabindeck_g(df):
    df.loc[df['CabinDeck']== 'G', 'HomePlanet'] = df.loc[df['CabinDeck'] == 'G', 'HomePlanet'].fillna('Earth')
    return df
def homeplanet_impute(df):
    df = fill_home_planet_by_group(df)
    df = fill_home_planet_by_last_name(df)
    df = home_planet_from_cabindeck_abc(df)
    df = home_planet_from_cabindeck_g(df)
    return df
togeth = homeplanet_impute(togeth)

In [46]:
def cabin_size(df):
    def cabin_size_apply(row):
        if pd.isna(row.Cabin):
            if row.GroupSize == 1:
                return 1
            else:
                return pd.NA
        else:
            samecabin = df[df.Cabin == row.Cabin]
            return len(samecabin)
    df['CabinSize'] = df.apply(cabin_size_apply, axis = 1)
    return df
togeth = cabin_size(togeth)

In [ ]:
"""idtocabin = {}
for i,row in togeth.iterrows():
    if not pd.isna(row.Cabin):
        idtocabin[row.PassengerId] = row.Cabin"""

In [ ]:
count = 0
for item in idtocabin.values():
    if item == 'G/2/S':
        count += 1
count

NameError: name 'idtocabin' is not defined

In [ ]:
import pickle
with open('idtocabin.pkl', 'wb') as fp:
    pickle.dump(idtocabin, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


In [25]:
togeth[togeth.Cabin.isna()].HomePlanet.value_counts()

HomePlanet
Earth     161
Europa     80
Mars       58
Name: count, dtype: int64

In [26]:
togeth[togeth.Cabin == 'G/2/S']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Set,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName,GroupSize


In [27]:
togeth.isna().sum()

PassengerId        0
HomePlanet        10
CryoSleep        310
Cabin            299
Destination      274
Age              270
VIP              296
RoomService      263
FoodCourt        289
ShoppingMall     306
Spa              284
VRDeck           268
Name             294
Set                0
Transported     4277
Group              0
GroupNumber        0
CabinDeck        299
CabinSide        299
CabinNum         299
FirstName        294
LastName         294
GroupSize          0
dtype: int64

In [28]:
def indexfromid(id):
    return togeth[togeth.PassengerId == id].index.item()
def idfromindex(index):
    return togeth.loc[index].PassengerId

In [29]:
def cabfill(cabfilllist):
    for item in cabfilllist:
        ind = indexfromid(item[0])
        togeth.at[ind,'Cabin'] = item[1]
    

In [30]:
togeth.at[indexfromid('3034_01'),'Cabin'] = 'B/98/P'
togeth.at[indexfromid('3053_01'),'Cabin'] = 'B/99/P'
togeth.at[indexfromid('4637_01'),'Cabin'] = 'E/300/S'
togeth.at[indexfromid('4652_01'),'Cabin'] = 'E/301/S'
togeth.at[indexfromid('9223_01'),'Cabin'] = 'F/1785/S'
togeth.at[indexfromid('9223_02'),'Cabin'] = 'F/1785/S'
togeth.at[indexfromid('5098_02'),'Cabin'] = 'G/832/S'


In [31]:
def caboptionstest():
    europadecks = list(togeth[togeth.HomePlanet == 'Europa'].CabinDeck.dropna().unique())
    earthdecks = list(togeth[togeth.HomePlanet == 'Earth'].CabinDeck.dropna().unique())
    marsdecks = list(togeth[togeth.HomePlanet == 'Mars'].CabinDeck.dropna().unique())
    sides = {'P':0,'S':0}
    side = ['P','S']
    caboptions = {}
    #cabin options
    for i,row in togeth.iterrows():
        if pd.isna(row.Cabin):
            bef = togeth.iloc[:i].dropna(subset=['Cabin'])
            aft = togeth.iloc[i+1:].dropna(subset=['Cabin'])
            if row.HomePlanet == 'Europa':
                decks = europadecks
            elif row.HomePlanet == 'Earth':
                decks = earthdecks
            else:
                decks = marsdecks
                
            for deck in decks:
                for sid in side:
                    before = bef[(bef.CabinDeck == deck) & (bef.CabinSide == sid)]
                    after = aft[(aft.CabinDeck == deck) & (aft.CabinSide == sid)]
                    if len(before) == 0:
                        minn = -2
                    else:
                        minn = max(list(before.CabinNum))
                    if len(after) == 0:
                        maxx = -2
                    else:
                        maxx = min(list(after.CabinNum))
                    if maxx - minn != 1 and maxx != 0:
                        if row.PassengerId in caboptions:
                            caboptions[row.PassengerId].append([minn,maxx,deck,sid])
                        else:
                            caboptions[row.PassengerId] = [[minn,maxx,deck,sid]]
    return caboptions

In [32]:
def caboptions():
    europadecks = list(togeth[togeth.HomePlanet == 'Europa'].CabinDeck.dropna().unique())
    earthdecks = list(togeth[togeth.HomePlanet == 'Earth'].CabinDeck.dropna().unique())
    marsdecks = list(togeth[togeth.HomePlanet == 'Mars'].CabinDeck.dropna().unique())
    sides = {'P':0,'S':0}
    side = ['P','S']
    caboptions = {}
    #cabin options
    for i,row in togeth.iterrows():
        if pd.isna(row.Cabin):
            bef = togeth.iloc[:i].dropna(subset=['Cabin'])
            aft = togeth.iloc[i+1:].dropna(subset=['Cabin'])
            if row.HomePlanet == 'Europa':
                decks = europadecks
            elif row.HomePlanet == 'Earth':
                decks = earthdecks
            else:
                decks = marsdecks
                
            for deck in decks:
                for sid in side:
                    before = bef[(bef.CabinDeck == deck) & (bef.CabinSide == sid)]
                    after = aft[(aft.CabinDeck == deck) & (aft.CabinSide == sid)]
                    if len(before) == 0:
                        minn = -2
                    else:
                        minn = max(list(before.CabinNum))
                    if len(after) == 0:
                        maxx = -2
                    else:
                        maxx = min(list(after.CabinNum))
                    if row.PassengerId in caboptions:
                        caboptions[row.PassengerId].append([minn,maxx,deck,sid])
                    else:
                        caboptions[row.PassengerId] = [[minn,maxx,deck,sid]]
    return caboptions

In [33]:
togeth[togeth.Group == 5098]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Set,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName,GroupSize
7125,5098_01,Earth,False,F/976/S,TRAPPIST-1e,44.0,False,0.0,83.0,0.0,68.0,586.0,Mistie Kellyons,Test,NaN,5098.0,1.0,F,S,976,Mistie,Kellyons,3
7126,5098_02,Earth,True,NaN,PSO J318.5-22,11.0,False,0.0,0.0,0.0,0.0,0.0,Sonald Kellyons,Test,NaN,5098.0,2.0,NaN,<NA>,<NA>,Sonald,Kellyons,3
7127,5098_03,Earth,True,G/832/S,55 Cancri e,2.0,False,0.0,0.0,0.0,0.0,0.0,Delody Kellyons,Test,NaN,5098.0,3.0,G,S,832,Delody,Kellyons,3


In [34]:
togethkeep = togeth.copy()

In [36]:
togeth = togethkeep.copy()

In [37]:

def cabfiller(df):
    def caboptionsbyplanet(planet,df):
        df = split_cabin(df)
        decks = list(df[df.HomePlanet == planet].CabinDeck.dropna().unique())
        sides = {'P':0,'S':0}
        side = ['P','S']
        caboptions = {}
        #cabin options
        for i,row in df[df.HomePlanet == planet].iterrows():
            if pd.isna(row.Cabin):
                bef = df.iloc[:i].dropna(subset=['Cabin'])
                aft = df.iloc[i+1:].dropna(subset=['Cabin'])
                for deck in decks:
                    for sid in side:
                        before = bef[(bef.CabinDeck == deck) & (bef.CabinSide == sid)]
                        after = aft[(aft.CabinDeck == deck) & (aft.CabinSide == sid)]
                        if len(before) == 0:
                            minn = -2
                        else:
                            minn = max(list(before.CabinNum))
                        if len(after) == 0:
                            maxx = -2
                        else:
                            maxx = min(list(after.CabinNum))
                        if row.PassengerId in caboptions:
                            caboptions[row.PassengerId].append([minn,maxx,deck,sid])
                        else:
                            caboptions[row.PassengerId] = [[minn,maxx,deck,sid]]
        return caboptions, df

    for planet in list(df.HomePlanet.dropna().unique()):
        
        planetcaboptions, df = caboptionsbyplanet(planet,df)

        def one_option_solo_group(indicaboptions):
        #only one option and solo group
            found = []
            for passenger in indicaboptions:
                if passenger == '0239_01'	:
                    print("'0239_01' first")
                if passenger == '0244_01':
                    print("'0244_01' first")
                count = 0
                for decks in indicaboptions[passenger]:
                    if decks[1] - decks[0] != 1:
                        count += 1
                    if decks[0] == -2 and decks[1] == 0:
                        count -= 1
                if len(df[df.Group == int(passenger[:4])]) == 1 and count == 1:
                    found.append(passenger)
            #which cabin is theres
            earthplug = []
            for passen in found:
                foundcabs = indicaboptions[passen]
                for item in foundcabs:
                    if item[1] - item[0] == 2 and item[0] != -2:
                        earthplug.append([passen, item[2] + "/" + str(item[0] + 1) +"/" + item[3]])
            for item in earthplug:
                if item[0] == '0239_01':
                    print("first 1")
                if item[0] == '0244_01':
                    print("first 2 ")
            #fill em
            cabfill(earthplug)
        one_option_solo_group(planetcaboptions)                                                                      
        

        def no_options(planetcaboptions):
        #no options
            found = []
            same = []
            for passenger in planetcaboptions:
                if passenger == '5098_02':
                    print("slay")
                if passenger == '0239_01'	:
                    print("'0239_01' second")
                if passenger == '0244_01':
                    print("'0244_01' second")
                count = 0
                for decks in planetcaboptions[passenger]:
                    if decks[1] - decks[0] > 1:
                        count += 1
                    if decks[0] == -2 and decks[1] == 0:
                        count -= 1
                if count == 0:
                    found.append(passenger)
                
            #which cabin is theres
            earthplug = []
            for passen in found:
                if len(list(df[df.Group == int(passen[:4])].Cabin.unique())) > 1:
                    cabs = list(df[df.Group == int(passen[:4])].Cabin.unique())
                    cabs = [x for x in cabs if not pd.isna(x)]
                    if len(cabs) > 1:
                        print(passen,cabs)
                    earthplug.append([passen,cabs[0]])          
            #fill em
            for item in earthplug:
                if item[0] == '0239_01':
                    print("")

                    print("second 1")
                if item[0] == '0244_01':
                    print("")

                    print("second 2 ")

            cabfill(earthplug)

        planetcaboptions, df = caboptionsbyplanet(planet,df)
    
        no_options(planetcaboptions)
    

        


    europacaboptions, df = caboptionsbyplanet('Europa',df)
    marscaboptions, df = caboptionsbyplanet('Mars',df)
    earthcaboptions, df = caboptionsbyplanet('Earth',df)

    indexlist = [indexfromid(x) for x in list(europacaboptions.keys())] + [indexfromid(x) for x in list(marscaboptions.keys())] + [indexfromid(x) for x in list(earthcaboptions.keys())]
    indexlist.sort()
    idlist = [idfromindex(x) for x in indexlist]
    globalcaboptions = {}
    for id in idlist:
        if id in europacaboptions:
            globalcaboptions[id] = europacaboptions[id]
        elif id in marscaboptions:
            globalcaboptions[id] = marscaboptions[id]
        else:
            globalcaboptions[id] = earthcaboptions[id]

    #only one that'll fit
    earthplug = []
    for i1,passenger in enumerate(globalcaboptions.keys()):
        if passenger == '0239_01':
            print("")
            print("'0239_01' third")
        if passenger == '0244_01':
            print("")
            print("'0244_01' third")
        for i2, cabs in enumerate(globalcaboptions[passenger]):
            if cabs[1] != 0 and cabs[1] == cabs[0] + 2:
                if passenger == '0239_01':
                    print('0239',"i2",i2,"cabs",cabs)
                if passenger == '0244_01':
                    print('0244',"i2",i2,"cabs",cabs)

                good = True
                low = i1 - 1
                high = i1  + 1
                lowchecking = True
                highchecking = True
                while lowchecking and low >= 0:
                    option = globalcaboptions[list(globalcaboptions.keys())[low]]
                    for choices in option:
                        if choices[2] == cabs[2] and choices[3] == cabs[3]:
                            lowchecking = False
                            if choices[1] >= cabs[0] + 1:
                                good = False
                    low -=1
                while highchecking and good and high <= len(globalcaboptions.keys()) -1 :
                    option = globalcaboptions[list(globalcaboptions.keys())[high]]
                    for choices in option:
                        if choices[2] == cabs[2] and choices[3] == cabs[3]:
                            highchecking = False
                            if choices[0] <= cabs[1] - 1:
                                good = False
                    high += 1
                if good:
                    earthplug.append([passenger,cabs[2] + "/" + str(cabs[0] + 1) + "/" + cabs[3]])
    for item in earthplug:
        if item[0] == '0239_01':
            print("third 1")
        if item[0] == '0244_01':
            print("third 2 ")
    cabfill(earthplug)

    return df


togeth = cabfiller(togeth)




    


    




    





slay
2425_05 ['G/396/P', 'F/495/P']
3287_02 ['G/522/S', 'F/621/S']
3712_01 ['G/606/S', 'F/701/S']
4005_04 ['G/657/P', 'F/829/P']
5098_02 ['F/976/S', 'G/832/S']
5601_03 ['G/904/S', 'F/1069/S']
6302_04 ['F/1305/P', 'G/1015/P']
6363_01 ['G/1026/P', 'E/405/P']
6405_02 ['F/1224/S', 'G/1039/S']
7890_03 ['F/1508/S', 'G/1273/S']
8129_01 ['G/1313/P', 'F/1683/P', 'E/522/P']
8728_07 ['F/1798/P', 'G/1416/P']
9081_03 ['G/1476/P', 'F/1858/P']
9238_05 ['G/1501/P', 'F/1890/P']
'0239_01' first
'0244_01' first
first 1
'0244_01' second
1709_03 ['F/326/S', 'D/61/S', 'E/127/S']
2092_03 ['D/70/S', 'E/153/S', 'F/410/S']
3411_02 ['E/232/S', 'F/645/S']

second 2 


In [ ]:
togeth.at[indexfromid('5098_02'),'Cabin'] = 'G/832/S'


In [40]:
togeth[togeth.Group == 2425]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Set,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName,GroupSize
3412,2425_01,Earth,True,G/396/P,TRAPPIST-1e,0.0,False,0.0,0.0,NaN,0.0,0.0,Raulia Tranklinay,Test,NaN,2425.0,1.0,G,P,396,Raulia,Tranklinay,6
3413,2425_02,Earth,False,G/396/P,PSO J318.5-22,28.0,False,0.0,NaN,1059.0,8.0,83.0,Trice Tranklinay,Test,NaN,2425.0,2.0,G,P,396,Trice,Tranklinay,6
3414,2425_03,Earth,False,F/495/P,TRAPPIST-1e,30.0,False,0.0,1307.0,848.0,246.0,0.0,Rickie Tranklinay,Test,NaN,2425.0,3.0,F,P,495,Rickie,Tranklinay,6
3415,2425_04,Earth,False,G/396/P,55 Cancri e,9.0,False,0.0,0.0,0.0,0.0,0.0,Elenna Tranklinay,Test,NaN,2425.0,4.0,G,P,396,Elenna,Tranklinay,6
3416,2425_05,Earth,True,G/396/P,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,Coley Tranklinay,Test,NaN,2425.0,5.0,G,P,396,Coley,Tranklinay,6
3417,2425_06,Earth,True,G/396/P,TRAPPIST-1e,4.0,False,0.0,0.0,0.0,0.0,0.0,Lan Tranklinay,Test,NaN,2425.0,6.0,G,P,396,Lan,Tranklinay,6


In [152]:

for cab in togeth.Cabin.unique():
    group = togeth[togeth.Cabin == cab]
    if len(group.Group.unique()) > 1:
        print(cab)


In [16]:
togeth[togeth.Cabin == 'G/2/S']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Set,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName,GroupSize,CabinSize
15,0012_01,Earth,False,G/2/S,TRAPPIST-1e,31.0,False,32.0,0.0,876.0,0.0,0.0,Justie Pooles,Train,False,12.0,1.0,G,S,2,Justie,Pooles,1,1


In [156]:
togeth.isna().sum()

PassengerId        0
HomePlanet        10
CryoSleep        310
Cabin             41
Destination      274
Age              270
VIP              296
RoomService      263
FoodCourt        289
ShoppingMall     306
Spa              284
VRDeck           268
Name             294
Set                0
Transported     4277
Group              0
GroupNumber        0
CabinDeck         41
CabinSide         41
CabinNum          41
FirstName        294
LastName         294
GroupSize          0
CabinSize        139
dtype: int64

In [80]:
togeth[togeth.PassengerId == '6060_01']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Set,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName,GroupSize
8465,6060_01,Mars,False,NaN,TRAPPIST-1e,37.0,False,1517.0,NaN,2104.0,0.0,147.0,Bleda Barte,Test,NaN,6060.0,1.0,NaN,<NA>,<NA>,Bleda,Barte,1


In [101]:
newdf = {}
for item in cabopts:
    newdf[item] = [cabopts[item]]
df = pd.DataFrame(newdf)
df = df.transpose()
df.to_csv('pdcd.csv')

In [100]:
df

,0
0293_01,"[[12, 14, B, P], [12, 14, C, S]]"
0310_01,"[[12, 14, B, P], [12, 14, C, S]]"
0348_02,"[[19, 22, E, P]]"
0364_02,"[[19, 22, E, P]]"
0374_02,"[[19, 22, E, P]]"
0992_04,"[[35, 35, D, S], [57, 59, E, P]]"
1006_03,"[[57, 59, E, P]]"
1011_01,"[[57, 59, E, P]]"
1041_01,"[[39, 41, C, S], [35, 37, D, S]]"
1095_01,"[[39, 41, C, S], [35, 37, D, S]]"


In [82]:
cabopts = caboptionstest()
cabopts['1041_01'].remove([57, 59, 'E', 'P'])
cabopts['6028_04'].remove([190, 192, 'D', 'P'])
cabopts['7463_01'].remove([1423, 1425, 'F', 'S'])
cabopts['7983_01'].remove([4, -2, 'T', 'P'])
cabopts['7983_01'].remove([3, -2, 'T', 'S'])
cabopts['7995_01'].remove([4, -2, 'T', 'P'])
cabopts['7995_01'].remove([3, -2, 'T', 'S'])

cabopts

{'0293_01': [[12, 14, 'B', 'P'], [12, 14, 'C', 'S']],
 '0310_01': [[12, 14, 'B', 'P'], [12, 14, 'C', 'S']],
 '0348_02': [[19, 22, 'E', 'P']],
 '0364_02': [[19, 22, 'E', 'P']],
 '0374_02': [[19, 22, 'E', 'P']],
 '0992_04': [[35, 35, 'D', 'S'], [57, 59, 'E', 'P']],
 '1006_03': [[57, 59, 'E', 'P']],
 '1011_01': [[57, 59, 'E', 'P']],
 '1041_01': [[39, 41, 'C', 'S'], [35, 37, 'D', 'S']],
 '1095_01': [[39, 41, 'C', 'S'], [35, 37, 'D', 'S']],
 '2513_01': [[518, 520, 'F', 'P'], [149, 151, 'E', 'P']],
 '2514_01': [[518, 520, 'F', 'P'], [149, 151, 'E', 'P']],
 '3598_01': [[589, 591, 'G', 'P'], [578, 580, 'G', 'S']],
 '3599_01': [[589, 591, 'G', 'P'], [578, 580, 'G', 'S']],
 '4953_01': [[192, 194, 'C', 'S'], [4, -2, 'T', 'P']],
 '4974_02': [[192, 194, 'C', 'S'], [4, -2, 'T', 'P']],
 '5480_01': [[213, 215, 'C', 'S'], [4, -2, 'T', 'P']],
 '5642_02': [[213, 215, 'C', 'S'], [4, -2, 'T', 'P']],
 '6028_04': [[57, 57, 'A', 'P'], [4, -2, 'T', 'P']],
 '6048_01': [[190, 192, 'D', 'P'], [386, 388, 'E', 'P']

In [108]:
togeth[togeth.Group == 1095]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Set,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName,GroupSize,CabinSize
1543,1095_01,Europa,True,NaN,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Alhail Drelcate,Train,True,1095.0,1.0,NaN,<NA>,<NA>,Alhail,Drelcate,1,1


In [106]:
togeth[(togeth.CabinDeck == 'G') & (togeth.CabinSide == 'P')].CabinSize.value_counts()

CabinSize
1    1292
2     222
3     150
5     100
4      72
6      60
7      21
8       8
Name: count, dtype: int64

In [89]:
togeth[(togeth.CabinDeck == 'D') & (togeth.CabinSide == 'P')].CabinSize.value_counts()

CabinSize
1    240
2     88
3     15
5     15
6     12
4      8
Name: count, dtype: int64

In [578]:
togeth[togeth.PassengerId == '9223_02']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Set,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName,GroupSize
12893,9223_02,Mars,True,F/1785/S,TRAPPIST-1e,17.0,False,0.0,0.0,0.0,0.0,0.0,Perit Sun,Test,NaN,9223.0,2.0,F,S,1785,Perit,Sun,2


In [553]:
togeth[togeth.PassengerId == '9070_01']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Set,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName,GroupSize
12671,9070_01,Earth,False,NaN,TRAPPIST-1e,20.0,False,323.0,0.0,542.0,0.0,0.0,Oraryn Flynney,Test,NaN,9070.0,1.0,NaN,<NA>,<NA>,Oraryn,Flynney,2


In [536]:
togeth[togeth.CabinDeck == 'G'].HomePlanet.value_counts()

HomePlanet
Earth    3869
Name: count, dtype: int64

In [326]:
togeth[togeth.Cabin.isna()].PassengerId

404      0293_01
421      0310_01
1466     1041_01
1543     1095_01
2421     1692_01
3529     2513_01
3530     2514_01
5016     3598_01
5017     3599_01
6493     4637_01
6514     4652_01
6927     4953_01
8450     6048_01
8465     6060_01
10081    7182_01
10082    7183_01
10313    7368_01
10394    7429_01
10408    7440_01
10434    7463_01
10440    7469_01
11082    7948_01
11129    7983_01
11148    7995_01
Name: PassengerId, dtype: object

In [552]:
def print_rows_with_different_group(df):
    unique_cabins = df['Cabin'].unique()
    
    for cabin in unique_cabins:
        group_list = df[df['Cabin'] == cabin]['Group'].unique()
        
        if len(group_list) > 1:
            print(cabin)

# Call the function with your DataFrame
print_rows_with_different_group(togeth)


In [537]:
sum = 0
remains = []
for cabinside in ['P','S']:
    for cabindeck in list(togeth.CabinDeck.unique()):
        setss = set(togeth[(togeth.CabinDeck == cabindeck) & (togeth.CabinSide == cabinside)].CabinNum)
        if len(setss) > 1:
            sum += len(set(range(min(setss),max(setss) + 1)).difference(setss))
            a = list((set(range(min(setss),max(setss) + 1))).difference(setss))
            a.sort()
            remains.append((cabindeck,cabinside,a))
            print(cabindeck,cabinside,a)
sum


B P [13, 98, 99]
F P [519, 1489, 1544]
A P [94]
G P [590, 1157, 1286, 1474]
C P []
E P [20, 21, 58, 150, 387]
D P [191, 235]
T P []
B S []
F S [1267, 1424, 1785]
A S []
G S [579, 1206, 1212, 1282]
C S [13, 40, 193, 214, 270, 298]
E S [300, 301, 528]
D S [36]
T S []


35

In [ ]:
togeth.at[indexfromid('3034_01'),'Cabin'] = 'B/98/P'
togeth.at[indexfromid('3053_01'),'Cabin'] = 'B/99/P'
togeth.at[indexfromid('9062_02'),'Cabin'] = 'F/1854/P'
togeth.at[indexfromid('9070_01'),'Cabin'] = 'G/1474/P'
togeth.at[indexfromid('9223_01'),'Cabin'] = 'F/1785/S'
togeth.at[indexfromid('9223_02'),'Cabin'] = 'F/1785/S'


In [546]:
cabopts = caboptions()
cabopts

{'0293_01': [[12, 14, 'B', 'P'],
  [7, 8, 'B', 'S'],
  [2, 3, 'A', 'P'],
  [3, 4, 'A', 'S'],
  [6, 7, 'C', 'P'],
  [12, 14, 'C', 'S'],
  [10, 11, 'D', 'P'],
  [10, 11, 'D', 'S'],
  [16, 17, 'E', 'P'],
  [17, 18, 'E', 'S'],
  [-2, 0, 'T', 'P'],
  [-2, 0, 'T', 'S']],
 '0310_01': [[12, 14, 'B', 'P'],
  [8, 9, 'B', 'S'],
  [3, 4, 'A', 'P'],
  [3, 4, 'A', 'S'],
  [7, 8, 'C', 'P'],
  [12, 14, 'C', 'S'],
  [11, 12, 'D', 'P'],
  [10, 11, 'D', 'S'],
  [16, 17, 'E', 'P'],
  [19, 20, 'E', 'S'],
  [-2, 0, 'T', 'P'],
  [-2, 0, 'T', 'S']],
 '0348_02': [[76, 77, 'F', 'P'],
  [64, 65, 'F', 'S'],
  [12, 13, 'D', 'P'],
  [10, 11, 'D', 'S'],
  [19, 22, 'E', 'P'],
  [21, 22, 'E', 'S']],
 '0364_02': [[81, 82, 'F', 'P'],
  [68, 69, 'F', 'S'],
  [12, 13, 'D', 'P'],
  [10, 11, 'D', 'S'],
  [19, 22, 'E', 'P'],
  [22, 23, 'E', 'S']],
 '0374_02': [[86, 87, 'F', 'P'],
  [68, 69, 'F', 'S'],
  [62, 63, 'G', 'P'],
  [53, 54, 'G', 'S'],
  [19, 22, 'E', 'P'],
  [22, 23, 'E', 'S']],
 '0992_04': [[37, 38, 'B', 'P'],
  [

In [543]:
ak = indexfromid(togeth[togeth.Cabin.isna()].iloc[-1].PassengerId)
togeth.iloc[ak -5 : ak + 5]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Set,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName,GroupSize
12888,9220_02,Earth,True,G/1496/P,TRAPPIST-1e,20.0,False,0.0,0.0,0.0,0.0,0.0,Frey Meyerthy,Train,True,9220.0,2.0,G,P,1496,Frey,Meyerthy,3
12889,9220_03,Earth,False,F/1886/P,PSO J318.5-22,22.0,False,623.0,0.0,NaN,0.0,53.0,Kennie Barrilloney,Train,True,9220.0,3.0,F,P,1886,Kennie,Barrilloney,3
12890,9221_01,Earth,True,G/1497/P,TRAPPIST-1e,21.0,False,0.0,0.0,0.0,0.0,NaN,Elicey Beachez,Train,False,9221.0,1.0,G,P,1497,Elicey,Beachez,1
12891,9222_01,Mars,True,F/1784/S,TRAPPIST-1e,50.0,False,0.0,0.0,0.0,NaN,0.0,Carves Chate,Train,True,9222.0,1.0,F,S,1784,Carves,Chate,1
12892,9223_01,Mars,True,NaN,TRAPPIST-1e,24.0,False,0.0,0.0,NaN,0.0,0.0,Weessh Sun,Test,NaN,9223.0,1.0,NaN,<NA>,<NA>,Weessh,Sun,2
12893,9223_02,Mars,True,NaN,TRAPPIST-1e,17.0,False,0.0,0.0,0.0,0.0,0.0,Perit Sun,Test,NaN,9223.0,2.0,NaN,<NA>,<NA>,Perit,Sun,2
12894,9224_01,Mars,False,F/1786/S,TRAPPIST-1e,24.0,False,745.0,639.0,977.0,4.0,0.0,Bants Reke,Train,False,9224.0,1.0,F,S,1786,Bants,Reke,1
12895,9225_01,Earth,False,F/1787/S,TRAPPIST-1e,45.0,False,1.0,197.0,0.0,660.0,15.0,Lynnon Meyersones,Train,False,9225.0,1.0,F,S,1787,Lynnon,Meyersones,1
12896,9226_01,Earth,False,F/1788/S,NaN,22.0,False,676.0,0.0,13.0,0.0,12.0,Stendy Connelson,Train,False,9226.0,1.0,F,S,1788,Stendy,Connelson,1
12897,9227_01,Earth,True,G/1498/P,TRAPPIST-1e,7.0,False,0.0,0.0,0.0,0.0,0.0,Glendy Hinglendez,Train,False,9227.0,1.0,G,P,1498,Glendy,Hinglendez,5


In [200]:
remains

[('A', 'P', []),
 ('B', 'P', [13, 98, 99]),
 ('C', 'P', []),
 ('D', 'P', [191, 235]),
 ('E', 'P', [20, 21, 58, 150, 387]),
 ('F', 'P', [152, 519, 1489, 1544]),
 ('G', 'P', [273, 590, 775, 1157, 1286, 1474]),
 ('T', 'P', []),
 ('A', 'S', []),
 ('B', 'S', []),
 ('C', 'S', [13, 40, 193, 270, 298]),
 ('D', 'S', [36]),
 ('E', 'S', [300, 301, 528]),
 ('F', 'S', [674, 1267, 1424, 1785]),
 ('G', 'S', [579, 1206, 1212, 1282]),
 ('T', 'S', [])]

In [547]:
togeth[togeth.HomePlanet == 'Mars'].CabinDeck.value_counts()

CabinDeck
F    1788
E     535
D     420
A       0
B       0
C       0
G       0
T       0
Name: count, dtype: int64

In [548]:
dict = {}
for item in list(cabopts.keys()):
    concern = togeth.loc[indexfromid(item)]
    if concern in dict:
        dict[concern] += 1
    else:
        dict[concern] = 1
dict

TypeError: unhashable type: 'Series'

In [549]:
cabopts

{'0293_01': [[12, 14, 'B', 'P'],
  [7, 8, 'B', 'S'],
  [2, 3, 'A', 'P'],
  [3, 4, 'A', 'S'],
  [6, 7, 'C', 'P'],
  [12, 14, 'C', 'S'],
  [10, 11, 'D', 'P'],
  [10, 11, 'D', 'S'],
  [16, 17, 'E', 'P'],
  [17, 18, 'E', 'S'],
  [-2, 0, 'T', 'P'],
  [-2, 0, 'T', 'S']],
 '0310_01': [[12, 14, 'B', 'P'],
  [8, 9, 'B', 'S'],
  [3, 4, 'A', 'P'],
  [3, 4, 'A', 'S'],
  [7, 8, 'C', 'P'],
  [12, 14, 'C', 'S'],
  [11, 12, 'D', 'P'],
  [10, 11, 'D', 'S'],
  [16, 17, 'E', 'P'],
  [19, 20, 'E', 'S'],
  [-2, 0, 'T', 'P'],
  [-2, 0, 'T', 'S']],
 '0348_02': [[76, 77, 'F', 'P'],
  [64, 65, 'F', 'S'],
  [12, 13, 'D', 'P'],
  [10, 11, 'D', 'S'],
  [19, 22, 'E', 'P'],
  [21, 22, 'E', 'S']],
 '0364_02': [[81, 82, 'F', 'P'],
  [68, 69, 'F', 'S'],
  [12, 13, 'D', 'P'],
  [10, 11, 'D', 'S'],
  [19, 22, 'E', 'P'],
  [22, 23, 'E', 'S']],
 '0374_02': [[86, 87, 'F', 'P'],
  [68, 69, 'F', 'S'],
  [62, 63, 'G', 'P'],
  [53, 54, 'G', 'S'],
  [19, 22, 'E', 'P'],
  [22, 23, 'E', 'S']],
 '0992_04': [[37, 38, 'B', 'P'],
  [

this one help?

In [204]:
togeth.at[indexfromid('3034_01'),'Cabin'] = 'B/98/P'
togeth.at[indexfromid('3053_01'),'Cabin'] = 'B/99/P'
togeth.at[indexfromid('9062_02'),'Cabin'] = 'F/1854/P'
togeth.at[indexfromid('9070_01'),'Cabin'] = 'G/1474/P'
togeth.at[indexfromid('9223_01'),'Cabin'] = 'F/1785/S'
togeth.at[indexfromid('9223_02'),'Cabin'] = 'F/1785/S'


In [170]:
akk =list(togeth[togeth.HomePlanet == 'Europa'].PassengerId)

In [172]:
for item in akk:
    if item in sdx:
        
        print(item, sdx[item])


0293_01 [[12, 14, 'B', 'P'], [7, 8, 'B', 'S'], [2, 3, 'A', 'P'], [3, 4, 'A', 'S'], [6, 7, 'C', 'P'], [12, 14, 'C', 'S'], [10, 11, 'D', 'P'], [10, 11, 'D', 'S'], [16, 17, 'E', 'P'], [17, 18, 'E', 'S'], [-2, 0, 'T', 'P'], [-2, 0, 'T', 'S']]
0310_01 [[12, 14, 'B', 'P'], [8, 9, 'B', 'S'], [3, 4, 'A', 'P'], [3, 4, 'A', 'S'], [7, 8, 'C', 'P'], [12, 14, 'C', 'S'], [11, 12, 'D', 'P'], [10, 11, 'D', 'S'], [16, 17, 'E', 'P'], [19, 20, 'E', 'S'], [-2, 0, 'T', 'P'], [-2, 0, 'T', 'S']]
0992_04 [[37, 38, 'B', 'P'], [40, 41, 'B', 'S'], [7, 8, 'A', 'P'], [10, 11, 'A', 'S'], [35, 36, 'C', 'P'], [38, 39, 'C', 'S'], [38, 39, 'D', 'P'], [35, 35, 'D', 'S'], [57, 59, 'E', 'P'], [73, 74, 'E', 'S'], [-2, 0, 'T', 'P'], [0, 1, 'T', 'S']]
1041_01 [[37, 38, 'B', 'P'], [40, 41, 'B', 'S'], [7, 8, 'A', 'P'], [12, 13, 'A', 'S'], [37, 38, 'C', 'P'], [39, 41, 'C', 'S'], [39, 40, 'D', 'P'], [35, 37, 'D', 'S'], [57, 59, 'E', 'P'], [74, 75, 'E', 'S'], [-2, 0, 'T', 'P'], [0, 1, 'T', 'S']]
1095_01 [[38, 39, 'B', 'P'], [42, 

In [174]:
for item in sdx:
    if item not in akk:
        print(item, sdx[item])

0348_02 [[76, 77, 'F', 'P'], [64, 65, 'F', 'S'], [12, 13, 'D', 'P'], [10, 11, 'D', 'S'], [19, 22, 'E', 'P'], [21, 22, 'E', 'S']]
0364_02 [[81, 82, 'F', 'P'], [68, 69, 'F', 'S'], [12, 13, 'D', 'P'], [10, 11, 'D', 'S'], [19, 22, 'E', 'P'], [22, 23, 'E', 'S']]
0374_02 [[86, 87, 'F', 'P'], [68, 69, 'F', 'S'], [62, 63, 'G', 'P'], [53, 54, 'G', 'S'], [19, 22, 'E', 'P'], [22, 23, 'E', 'S']]
1006_03 [[205, 206, 'F', 'P'], [187, 188, 'F', 'S'], [147, 148, 'G', 'P'], [156, 157, 'G', 'S'], [57, 59, 'E', 'P'], [74, 75, 'E', 'S']]
1011_01 [[206, 207, 'F', 'P'], [188, 189, 'F', 'S'], [147, 148, 'G', 'P'], [157, 158, 'G', 'S'], [57, 59, 'E', 'P'], [74, 75, 'E', 'S']]
2513_01 [[518, 520, 'F', 'P'], [483, 484, 'F', 'S'], [405, 406, 'G', 'P'], [404, 405, 'G', 'S'], [149, 151, 'E', 'P'], [176, 177, 'E', 'S']]
2514_01 [[518, 520, 'F', 'P'], [483, 484, 'F', 'S'], [405, 406, 'G', 'P'], [404, 405, 'G', 'S'], [149, 151, 'E', 'P'], [176, 177, 'E', 'S']]
3598_01 [[745, 746, 'F', 'P'], [677, 678, 'F', 'S'], [589